<a href="https://colab.research.google.com/github/arva29/AMDproject/blob/main/UkraineWarTweetsProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this is meant to be run on google colab

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7") #SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [ ]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

100% 12.5G/12.5G [02:08<00:00, 111MB/s]
100% 12.5G/12.5G [02:08<00:00, 104MB/s]


In [ ]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0819_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0820_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0821_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0822_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0823_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0824_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0825_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0826_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0827_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0828_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0829_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0830_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0831_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0901_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0902_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0903_UkraineCombinedTweetsDeduped.csv.gzip  
  inflat

I extract the collection of tweets from the (23 of September ?? ) and I will work on this.

In [ ]:
import csv
import gzip

data_list = []

with gzip.open('1008_UkraineCombinedTweetsDeduped.csv.gzip', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data_list.append(row)

rdd = sc.parallelize(data_list)

In [ ]:
rdd.count()

In [ ]:
rdd.first()

OrderedDict([('', '130404'),
             ('userid', '1603709857'),
             ('username', 'trajanpublisher'),
             ('acctdesc',
              "We are Trajan Media, owner of Canada's premier numismatic and philatelic publications –\xa0Canadian Coin News and Canadian Stamp News –\xa0plus Coin & Stamp Supplies."),
             ('location', 'Canada'),
             ('following', '190'),
             ('followers', '916'),
             ('totaltweets', '7215'),
             ('usercreatedts', '2013-07-18 15:23:57'),
             ('tweetid', '1578535646146199554'),
             ('tweetcreatedts', '2022-10-08 00:00:00'),
             ('retweetcount', '1'),
             ('text',
              'Canada joined 15 countries whose flags grace the obverse of Ukraine’s first commemorative coin issued since the beginning of Russia’s renewed invasion this February.\n\nRead more: https://t.co/vRhROceH9x\n\n#numismatics #Ukraine️ #collectcoins'),
             ('hashtags',
              "[{'text':

Tokenize tweet by removing puntuation and stop words in order to have only meaningful words to understand the meaning of the tweet. I also put all the words lower case and remove the link at the end of the tweets.

In [ ]:
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 21.9 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=45edcd8c3d3e850ec6a4970c6088d4acbfdc3190effbc518f5221bce2d043b9c
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from cleantext import clean

nltk.download('stopwords')
stopw = set(stopwords.words('english'))

tokenizer = TweetTokenizer()

# Without stopwords
#tokenize = lambda text: [word for word in tokenizer.tokenize(clean(text, no_urls=True, no_emails=True, no_emoji=True, no_line_breaks=True, replace_with_email="", replace_with_url="")) if word not in punctuation and word not in stopw]

# With stopwords
tokenize = lambda text: [word for word in tokenizer.tokenize(clean(text, no_urls=True, no_emails=True, no_emoji=True, no_line_breaks=True, replace_with_email="", replace_with_url="")) if word not in punctuation]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import numpy as np

def sort(list):
  list.sort()
  return list

en_tweets = rdd.filter(lambda x: x.get("language")=='en').map(lambda x: (x, x.get("text")))

tweets_tokenized = en_tweets.map(lambda x: (x[0].get("tweetid"), tokenize(x[1])))

tweets_tokenized.first()

('1578535646146199554',
 ['canada',
  'joined',
  '15',
  'countries',
  'whose',
  'flags',
  'grace',
  'the',
  'obverse',
  'of',
  "ukraine's",
  'first',
  'commemorative',
  'coin',
  'issued',
  'since',
  'the',
  'beginning',
  'of',
  "russia's",
  'renewed',
  'invasion',
  'this',
  'february',
  'read',
  'more',
  '#numismatics',
  '#ukraine',
  '#collectcoins'])

## *K*-Shingles

In [ ]:
K = int(input("Enter the number K for k-grams dimension: "))

def build_shingles(tokens, K):
  shingles = list()
  tmp = ""

  for i in range(len(tokens)):
    tmp = tokens[i]
    if((len(tokens) - (i)) < K):
      break;
    for k in range(K - 1):
      tmp += " " + tokens[i + (k + 1)]
    shingles.append(tmp)

  return shingles

shingles_rdd = tweets_tokenized.map(lambda x: (x[0], sort(build_shingles(x[1], K))))
shingles_rdd.first()

Enter the number K for k-grams dimension: 3


('1578535646146199554',
 ['#numismatics #ukraine #collectcoins',
  '15 countries whose',
  "beginning of russia's",
  'canada joined 15',
  'coin issued since',
  'commemorative coin issued',
  'countries whose flags',
  'february read more',
  'first commemorative coin',
  'flags grace the',
  'grace the obverse',
  'invasion this february',
  'issued since the',
  'joined 15 countries',
  'more #numismatics #ukraine',
  "obverse of ukraine's",
  "of russia's renewed",
  "of ukraine's first",
  'read more #numismatics',
  'renewed invasion this',
  "russia's renewed invasion",
  'since the beginning',
  'the beginning of',
  'the obverse of',
  'this february read',
  "ukraine's first commemorative",
  'whose flags grace'])

Cast from list to set and then to list again in order to remove all the duplicate elements

In [ ]:
shingles_bag = sort(list(set(shingles_rdd.flatMap(lambda x: x[1]).collect())))
shingles_bag[0:10]

['##biggerthanme #osimhen #dazn',
 '##crimea #tma2022 #thefactmusicawards2022',
 '##crimea as #ukraine',
 '##dpr allied forces',
 '##droz #democraticviolence #gpexplorer',
 '##iranianlivesmatter #iranprotests2022 #tv3newday',
 '##kimjongun and #danielortega',
 '##kpworldtoursingapore #kerch #weverse',
 '##marvel #ironman #thor',
 '##nctdream #pakistanzindabad #case143']

I use integers from 0 to *n* (with n equal to the length of shgles_bag) to store the shingles list

In [ ]:
shingles_bag_id = list(range(0, len(shingles_bag)))
len(shingles_bag_id)

572145

In [ ]:
def convertShingles(shingles, BoS):
  listToReturn = [0]*len(BoS)
  start_index = 0;

  for s in shingles:
    for i in range(start_index, len(BoS)):
      if(s == BoS[i]):
        listToReturn[i] = 1
        start_index = i
        break;

  return listToReturn

shingles_int_rdd = shingles_rdd.map(lambda x: (x[0], convertShingles(x[1], shingles_bag)))
#shingles_int_rdd.first()

Hashing function for permutation

In [ ]:
def generatePermutation(arrayToHash, a, b):
  arrayToReturn = arrayToHash.copy()
  
  for i in range(len(arrayToHash)):
    arrayToReturn[i] = (a * arrayToHash[i] + b) % len(arrayToHash)

  return arrayToReturn

n = int(input("Insert number n of permutation desired: "))
permutation_matrix = list()

for i in range(n):
  #Generate 2 random numbers in order to have different hashing by randomly chaging only these two numbers
  a = np.random.randint(1000) 
  b = np.random.randint(1000)
  #print(a, b)

  permutation_matrix.append(generatePermutation(shingles_bag_id, a, b))
  #print(permutation_matrix[i][:30])

Insert number n of permutation desired: 10


In [ ]:
def calculateMinhashSignature(perm_mtx, shingle_array):
  sig_column = [np.inf] * len(perm_mtx)

  for i in range(len(shingle_array)):
    if(shingle_array[i] == 1):
      for j in range(len(sig_column)):
        if(perm_mtx[j][i] < sig_column[j]):
          sig_column[j] = perm_mtx[j][i]

  return sig_column

minhash_sig_rdd = shingles_int_rdd.map(lambda x: (x[0], calculateMinhashSignature(permutation_matrix, x[1])))
minhash_sig_rdd.take(10)

[('1578535646146199554',
  [9400, 6009, 32616, 434, 104005, 3699, 21690, 28338, 10035, 18091]),
 ('1578535651984650240',
  [50652, 11523, 71706, 1899, 97051, 6659, 20245, 12128, 35965, 42463]),
 ('1578535652248850432',
  [160179, 2847, 43961, 26264, 83581, 15714, 14311, 23464, 49945, 1114]),
 ('1578535654149038080',
  [12027, 65733, 26391, 108454, 14695, 3748, 243068, 17376, 15995, 86638]),
 ('1578535658691629056',
  [61506, 3522, 14936, 5684, 5893, 34505, 16264, 11078, 1440, 11194]),
 ('1578535679713497089',
  [2493, 15150, 27076, 5239, 2743, 2595, 13129, 2563, 14175, 15628]),
 ('1578535680585916417',
  [3613, 18600, 9671, 38239, 12229, 2675, 7576, 10103, 10505, 19840]),
 ('1578535686818304000',
  [1088, 876, 26146, 10769, 1147, 3775, 1283, 3879, 12805, 37270]),
 ('1578535704694779907',
  [103050, 1830, 94061, 507499, 323395, 160810, 16770, 147275, 83415, 349705]),
 ('1578535714945658882',
  [175, 28038, 17256, 299, 7309, 22374, 50577, 725, 11825, 64930])]

In [ ]:
b = int(input("Enter into how many bands you want to divide the " + str(n) +" permutations: "))

def band_splitting(tweet_id, arrayToSplit, b, n):
  tmp = list()
  arrayToReturn = list()
  band_id = 1

  r = int(n/b)
  i = 0

  while i < len(arrayToSplit):
    for j in range(r):
      tmp.append(arrayToSplit[i + j])
    
    i += r
    arrayToReturn.append((tweet_id, (band_id, tmp.copy())))
    band_id += 1
    tmp.clear()

  return arrayToReturn

sig_bands_rdd = minhash_sig_rdd.flatMap(lambda x: band_splitting(x[0], x[1], b, n))
sig_bands_rdd.take(10)

In [ ]:
import hashlib

def hashArray(array):
  col_str = ' '.join(str(x) for x in array)

  return hash(col_str)
  #return int(hashlib.sha1(col_str.encode('utf-8')).hexdigest(), 16)

sig_bands_hashed_rdd = sig_bands_rdd.map(lambda x: ((hashArray(x[1][1]), x[1][0]), x[0]))
sig_bands_hashed_rdd.persist()
#sig_bands_hashed_rdd.count()

PythonRDD[27] at RDD at PythonRDD.scala:53

In [ ]:
sig_bands_hashed_rdd.take(100)

KeyboardInterrupt: ignored

# DEBUG

In [ ]:
def generatePermutation(arrayToHash, a, b):
  arrayToReturn = arrayToHash.copy()
  
  for i in range(len(arrayToHash)):
    arrayToReturn[i] = (a * arrayToHash[i] + b) % len(arrayToHash)

  #print(arrayToHash, a, b, arrayToReturn)

  return arrayToReturn
  
def calculateMinhashSignature(perm_mtx, shingle_array):
  sig_column = [np.inf] * len(perm_mtx)

  for i in range(len(shingle_array)):
    if(shingle_array[i] == 1):
      for j in range(len(sig_column)):
        if(perm_mtx[j][i] < sig_column[j]):
          #print(perm_mtx[j][i])
          sig_column[j] = perm_mtx[j][i]

  return sig_column

n = int(input("Insert number n of permutation desired: "))
permutation_matrix = list()
tmp = list(range(0, 10))
array = [0,0,0,0,1,0,0,1,1,0]

for i in range(n):
  #Generate 2 random numbers in order to have different hashing by randomly chaging only these two numbers
  a = np.random.randint(10) 
  b = np.random.randint(10)

  permutation_matrix.append(generatePermutation(tmp, a, b))
  
#[print(*line) for line in permutation_matrix]
  
a = calculateMinhashSignature(permutation_matrix, array)
print(a)


Insert number n of permutation desired: 10
[3, 1, 1, 6, 2, 4, 1, 1, 0, 4]


In [ ]:
def band_splitting(tweet_id, arrayToSplit, b, n):
  tmp = list()
  arrayToReturn = list()
  band_id = 1

  r = int(n/b)
  i = 0
  while i < len(arrayToSplit):
    print(i)
    for j in range(r):
      tmp.append(arrayToSplit[i + j])
    i += r
    print(tmp)
    arrayToReturn.append((tweet_id, (band_id, tmp.copy())))
    band_id += 1
    tmp.clear()

  return arrayToReturn

b = 5
ex = band_splitting("xx", a, b, 10)
ex

0
[3, 1]
2
[1, 6]
4
[2, 4]
6
[1, 1]
8
[0, 4]


[('xx', (1, [3, 1])),
 ('xx', (2, [1, 6])),
 ('xx', (3, [2, 4])),
 ('xx', (4, [1, 1])),
 ('xx', (5, [0, 4]))]

In [ ]:
import hashlib

a = "cnoefvrivpvòslkfdvmkfvpòfvevflàeràòwwwwedweferfeev"

#b = int(hashlib.sha1(str(a).encode('utf-8')).hexdigest(), 16)
b = hashlib.sha1(str(a).encode('utf-8')).hexdigest()
#int(hashlib.sha1(str(a).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)

b

'75c93d49c4b7c23b5ba3abde96216cfe2355bd82'

In [ ]:
#a = [1,2,3,4,5,6,7,8]
a = ['a','b','c','d']

b = ' '.join(str(x) for x in a)

b

'a b c d'

## TO DO:
- Hashing list on shingles
- Verify if the hash is worth in terms of space 